# Preprocessing script
Author: Josefine Zerbe  
Date: 21.03.2022

In [1]:
import os
from os import pardir
from os.path import abspath
from os.path import join as pjoin

import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import bids
from matplotlib.gridspec import GridSpec
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.image import load_img, threshold_img, math_img, resample_to_img
from nilearn.masking import intersect_masks
from nipype import config
config.enable_debug_mode()
from nipype.interfaces import fsl
from nipype.interfaces.fsl.maths import TemporalFilter
from nipype.interfaces.fsl.model import MELODIC
from nipype.interfaces.fsl.preprocess import SUSAN
from nipype.interfaces.utility import Function
from nipype.pipeline.engine import MapNode, Workflow, Node
from numpy.random import choice as rndchoice
from scipy.ndimage.morphology import binary_erosion
from scipy.signal import periodogram
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from bids import BIDSLayout

/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()
/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


220413-22:42:24,158 nipype.utils WARNING:
	 A newer version (1.7.1) of nipy/nipype is available. You are using 1.7.0


In [36]:
'nipype.interfaces.fsl.preprocess' in sys.modules

True

In [2]:
DATASET_PATH = '/LOCAL/jzerbe/faces_vs_houses' # bidsroot
RAWDATA_PATH = '/LOCAL/jzerbe/faces_vs_houses/ds002938' # 
PREPROCESSED_PATH = '/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep' # fmriprepdir
OUTPUT_PATH = '/LOCAL/jzerbe/faces_vs_houses/derivatives/melodic' # 

SUBJECT = 'sub-03' # subject
BOLDFILE = 'task-effort_space-T1w_desc-preproc_bold.nii.gz'
MASK = 'task-effort_space-T1w_desc-brain_mask.nii.gz'
OUTPUT = 'task-effort_space-T1w_melodic'

In [3]:
def calc_susan_thresh(boldfile, maskfile, timeax=0, median_factor=.75):
    """
    Calculate the median value within brainmask and multiply with fixed factor to get an estimate of the contrast
    between background and brain for FSL's SUSAN.
    """
    from nilearn.masking import apply_mask
    import numpy as np
    data = apply_mask(boldfile, maskfile)
    med = np.median(data.mean(axis=timeax))
    del data  # suspect memory leak
    return med * median_factor

In [4]:
def run_melodic_wf(
        subject, bidsroot, fmriprepdir, bold_file, mask_file, outdir,
        space='T1w',
        approach: str = 'runwise',
        fwhm: float = 4., hpf_sec: float = 120.,
        derivname: str = 'melodic',
        nprocs: int = 30, melodic_mem: float = 400.,
        tr: float = 1.5, try_last_n_runs: bool or int = False,
):
    """
    Run Melodic on the preprocessed functional images.
    Besides fwhm and hpf for additional preprocessing, user can choose reference space (func_preproc or T1w) and
    approach (runwise or concat). Note that concat requires data to be in T1w space.
    Example:
        import sys
        subject = sys.argv[1]
        bidsroot = abspath(pjoin(pardir, pardir, pardir))
        fmriprepdir = pjoin(bidsroot, 'derivatives', 'fmriprep', f'sub-{subject}')
        run_melodic_wf(
            subject, bidsroot, fmriprepdir, approach='runwise', space='T1w',
        )
    """
    wf_bdir = pjoin(bidsroot, 'melodicwf_wdir', f'space-{space}', subject)
    derivdir = pjoin(bidsroot, 'derivatives', derivname, approach)
    for d in [wf_bdir, derivdir]:
        if not os.path.exists(d):
            os.makedirs(d)
    boldfiles = pjoin(fmriprepdir, subject, 'func', f'{subject}_{bold_file}')
    masks = pjoin(fmriprepdir, subject, 'func', f'{subject}_{mask_file}')
    outdirs = pjoin(derivdir, subject, f'space-T1w', f'{subject}_{outdir}')
    #print('boldfiles: ', boldfiles, '\n',
    #      'masks: ', masks, '\n',
    #      'outdirs: ', outdirs, '\n')
    
    if try_last_n_runs:
        boldfiles = boldfiles[-try_last_n_runs:]
        masks = masks[-try_last_n_runs:]
        outdirs = outdirs[-try_last_n_runs:]

    wf = Workflow(name='melodicwf',
                  base_dir=wf_bdir)
    calcthresh = MapNode(Function(function=calc_susan_thresh,
                                  input_names=['boldfile', 'maskfile'], output_names=['smooth_thresh']),
                         name='calcthresh', iterfield=['boldfile', 'maskfile'])

    calcthresh.inputs.boldfile = boldfiles
    calcthresh.inputs.maskfile = masks
    susan = MapNode(SUSAN(fwhm=fwhm), iterfield=['in_file', 'brightness_threshold'], name='susan')
    susan.inputs.in_file = boldfiles
    tfilt = MapNode(TemporalFilter(highpass_sigma=float(hpf_sec / tr)), iterfield=['in_file'], name='tfilt')
    calcthresh.inputs.maskfile = masks
    if approach == 'runwise':
        melodic = MapNode(
            MELODIC(tr_sec=tr, out_all=True, no_bet=True, report=True), iterfield=['in_files', 'mask', 'out_dir'],
            name='melodic_runwise')
        melodic.inputs.mask = masks
        melodic.inputs.out_dir = outdirs
    elif approach == 'concat':
        melodic = Node(MELODIC(tr_sec=tr, out_all=True, no_bet=True, report=True, approach='concat', args='--debug'),
                       name='melodic_concat', mem_gb=melodic_mem, n_procs=nprocs)
        outdir = abspath(pjoin(derivdir, f'space-{space}', f'sub-{subject}', 'concat'))
        if not os.path.exists(outdir):
            os.makedirs(outdir)
        umask_img = intersect_masks(masks, threshold=0)
        umask_img.to_filename(pjoin(wf_bdir, 'umask.nii.gz'))
        melodic.inputs.out_dir = outdir
        melodic.terminal_output = 'stream'
        melodic.inputs.mask = pjoin(wf_bdir, 'umask.nii.gz')
    else:
        raise ValueError(f'"approach" must be in ["runwise", "concat"]')
    wf.connect([
        (calcthresh, susan, [('smooth_thresh', 'brightness_threshold')]),
        (susan, tfilt, [('smoothed_file', 'in_file')]),
        (tfilt, melodic, [('out_file', 'in_files')]),
    ])
    wf.run() #plugin='MultiProc', plugin_args=dict(n_procs=nprocs))
    

run_melodic_wf(SUBJECT, DATASET_PATH, PREPROCESSED_PATH, BOLDFILE, MASK, OUTPUT)



220413-22:42:36,114 nipype.workflow DEBUG:
	 adding multipath trait: boldfile
220413-22:42:36,121 nipype.workflow DEBUG:
	 adding multipath trait: maskfile
220413-22:42:36,122 nipype.workflow DEBUG:
	 setting mapnode(calcthresh) input: boldfile -> ['/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-preproc_bold.nii.gz']
220413-22:42:36,122 nipype.workflow DEBUG:
	 setting mapnode(calcthresh) input: maskfile -> ['/LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-brain_mask.nii.gz']
220413-22:42:36,123 nipype.interface WARNING:
	 FSLOUTPUTTYPE environment variable is not set. Setting FSLOUTPUTTYPE=NIFTI
220413-22:42:36,124 nipype.workflow DEBUG:
	 adding multipath trait: brightness_threshold
220413-22:42:36,125 nipype.workflow DEBUG:
	 adding multipath trait: in_file
220413-22:42:36,126 nipype.workflow DEBUG:
	 setting mapnode(susan) input: in_file -> ['/LOCAL/jzerbe/faces_vs_houses/derivatives/

220413-22:42:36,308 nipype.workflow DEBUG:
	 adding multipath trait: brightness_threshold
220413-22:42:36,309 nipype.workflow DEBUG:
	 adding multipath trait: in_file
220413-22:42:36,309 nipype.workflow DEBUG:
	 [MapNode] Resume - hashfile=/LOCAL/jzerbe/faces_vs_houses/melodicwf_wdir/space-T1w/sub-03/melodicwf/susan/_0xf9814579cf8d8f199659b612f10546bd_unfinished.json
220413-22:42:36,312 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/LOCAL/jzerbe/faces_vs_houses/melodicwf_wdir/space-T1w/sub-03/melodicwf/susan/_report/report.rst"
220413-22:42:36,318 nipype.workflow DEBUG:
	 setting input 0 in_file /LOCAL/jzerbe/faces_vs_houses/derivatives/fmriprep/sub-03/func/sub-03_task-effort_space-T1w_desc-preproc_bold.nii.gz
220413-22:42:36,320 nipype.workflow DEBUG:
	 setting input 0 brightness_threshold 557.1193842626928
220413-22:42:36,321 nipype.workflow INFO:
	 [Node] Setting-up "_susan0" in "/LOCAL/jzerbe/faces_vs_houses/melodicwf_wdir/space-T1w/sub-03/melodicwf/susan/mapflow/_sus

NodeExecutionError: Exception raised while executing Node _susan0.

Traceback (most recent call last):
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/interfaces/base/core.py", line 398, in run
    runtime = self._run_interface(runtime)
  File "/LOCAL/jzerbe/testenv/lib/python3.6/site-packages/nipype/interfaces/base/core.py", line 736, in _run_interface
    % (executable_name, runtime.hostname)
OSError: No command "susan" found on host nerz. Please check that the corresponding package is installed.


In [56]:
def get_obs(datasetdir):
    
    return niifiles

def get_data(subject):
    
    return 


def grab_data(subject, bidsroot, fmriprepdir, derivdir, space='func_preproc'):
    ds = ThingsMRIdataset(bidsroot)
    bidsobjs = ds.layout.get(subject=subject, suffix='bold', extension='.nii.gz')
    space_str = {'T1w': '_space-T1w', 'func_preproc': ''}
    assert space in space_str
    boldfiles = [abspath(pjoin(
        fmriprepdir, f"ses-{bidsobj.entities['session']}", 'func',
        f"sub-{subject}_ses-{bidsobj.entities['session']}_task-{bidsobj.entities['task']}_run-{bidsobj.entities['run']}{space_str[space]}_desc-preproc_bold.nii.gz"
    )) for bidsobj in bidsobjs]
    masks = [abspath(pjoin(
        fmriprepdir, f"ses-{bidsobj.entities['session']}", 'func',
        f"sub-{subject}_ses-{bidsobj.entities['session']}_task-{bidsobj.entities['task']}_run-{bidsobj.entities['run']}{space_str[space]}_desc-brain_mask.nii.gz"
    )) for bidsobj in bidsobjs]
    outdirs = [abspath(pjoin(
        derivdir, f'space-{space}', f'sub-{subject}', f"ses-{bidsobj.entities['session']}",
        f"sub-{subject}_ses-{bidsobj.entities['session']}_task-{bidsobj.entities['task']}_run-{bidsobj.entities['run']}_melodic"))
        for bidsobj in bidsobjs]
    for o in outdirs:
        if not os.path.exists(o):
            os.makedirs(o)
    return boldfiles, masks, outdirs


def calc_hfc(timeseries, tr=1.5):
    """Calculate high frequency content for time series data. Tr can generally mean sampling rate in seconds."""
    nf = (1. / tr) * .5  # nyquist
    freqs, power = periodogram(timeseries, fs=1. / tr)
    relcumsum = np.cumsum(power) / power.sum()
    freqind = np.argmin(np.absolute(relcumsum - .5))
    hfc = freqs[freqind] / nf
    return hfc
